# Eigenface example

In [1]:
import numpy as np 
from PIL import Image
import os

In [2]:
path = 'unpadded/'  
ids = range(1, 16)   
states = ['centerlight', 'glasses', 'happy', 'leftlight', 
          'noglasses', 'normal', 'rightlight','sad', 
          'sleepy', 'surprised', 'wink']
prefix = 'subject'
suffix = '.jpg'

# Lay anh dau tien lam vi du
fn = os.path.join(path, f'{prefix}01.{states[0]}{suffix}')

# Doc anh bang PIL
# chuyen sang anh grayscale
img = Image.open(fn).convert('L')  
# resize ve dung kich thuoc
img = img.resize((98, 116))     
# chuyen thanh numpy array   
img_array = np.array(img)          

h = 116  
w = 98   
D = h * w
N = len(states) * 15

X = np.zeros((D, N))
X[:, 0] = img_array.reshape(D)

FileNotFoundError: [Errno 2] No such file or directory: 'unpadded/subject01.centerlight.jpg'

In [ ]:
# collect all data
from PIL import Image

cnt = 0 
for person_id in range(1, 16):
    for state in states:
        fn = path + prefix + str(person_id).zfill(2) + '.' + state + suffix
        try:
            # chuyen sang anh xam
            img = Image.open(fn).convert('L')  
            # resize ve kich thuoc 98x116     
            img = img.resize((w, h))           
            # dua anh ve vector 1 chieu     
            X[:, cnt] = np.array(img).reshape(D)    
            cnt += 1
        except Exception as e:
            print(f'Loi khi xu ly {fn}: {e}')


In [ ]:
# Doing PCA, note that each row is a datapoint
from sklearn.decomposition import PCA
pca = PCA(n_components=100) # k = 100 
pca.fit(X.T)

In [ ]:
# projection matrix 
U = pca.components_.T

In [ ]:
import matplotlib.pyplot as plt
for i in range(U.shape[1]):
    plt.axis('off')
    f1 = plt.imshow(U[:, i].reshape(116, 98), interpolation='nearest')
    f1.axes.get_xaxis().set_visible(False)
    f1.axes.get_yaxis().set_visible(False)
#     f2 = plt.imshow(, interpolation='nearest' )
    plt.gray()
    fn = 'eigenface' + str(i).zfill(2) + '.png'
    plt.savefig(fn, bbox_inches='tight', pad_inches=0)
#    plt.show()
    


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# See reconstruction of first 6 persons 
for person_id in range(1, 7):
    for state in ['centerlight']:
        fn = path + prefix + str(person_id).zfill(2) + '.' + state + suffix
        img = Image.open(fn).convert('L')
        img = img.resize((98, 116))  
        im = np.array(img)

        # HIỂN THỊ ẢNH GỐC
        plt.figure()
        plt.axis('off')
        f1 = plt.imshow(im, interpolation='nearest')
        plt.text(5, 10, 'Ảnh gốc', fontsize=10, color='red', backgroundcolor='white')
        fn = 'ori' + str(person_id).zfill(2) + '.png'
        plt.savefig(fn, bbox_inches='tight', pad_inches=0)
        plt.show()

        # PCA: reshape và trừ mean
        x = im.reshape(D, 1) - pca.mean_.reshape(D, 1)
        # encode
        z = U.T.dot(x)  
        # decode
        x_tilde = U.dot(z) + pca.mean_.reshape(D, 1) 

        # reshape về kích thước ảnh gốc
        im_tilde = x_tilde.reshape(116, 98)

        # HIỂN THỊ ẢNH TÁI DỰNG
        plt.figure()
        plt.axis('off')
        f2 = plt.imshow(im_tilde, interpolation='nearest')
        plt.text(5, 10, 'Tái dựng', fontsize=10, color='blue', backgroundcolor='white')
        fn = 'res' + str(person_id).zfill(2) + '.png'
        plt.savefig(fn, bbox_inches='tight', pad_inches=0)
        plt.show()


In [ ]:
# HIỂN THỊ VECTOR ĐÃ ENCODE (z) - chỉ để trực quan hóa
plt.figure()
plt.axis('off')
z_img = z.reshape(10, 10)  # reshape (100, 1) thành (10, 10) cho dễ nhìn
plt.imshow(z_img, cmap='gray', interpolation='nearest')
plt.text(1, 1, 'Đã encode (z)', fontsize=10, color='green', backgroundcolor='white')
fn = 'enc' + str(person_id).zfill(2) + '.png'
plt.savefig(fn, bbox_inches='tight', pad_inches=0)
plt.show()


In [ ]:
# Tính sai số tái dựng (MSE) cho 6 người đầu tiên
reconstruction_errors = []

for person_id in range(1, 7):
    for state in ['centerlight']:
        fn = path + prefix + str(person_id).zfill(2) + '.' + state + suffix
        img = Image.open(fn).convert('L')
        img = img.resize((98, 116))
        im = np.array(img)

        x = im.reshape(D, 1) - pca.mean_.reshape(D, 1)
        z = U.T.dot(x)
        x_tilde = U.dot(z) + pca.mean_.reshape(D, 1)

        # Tính sai số MSE
        mse = np.mean((x + pca.mean_.reshape(D, 1) - x_tilde) ** 2)
        reconstruction_errors.append(mse)

# Vẽ biểu đồ sai số
plt.figure(figsize=(8, 4))
plt.plot(range(1, 7), reconstruction_errors, marker='o', color='b')
plt.title("Sai số tái dựng (MSE) cho từng người (PCA)")
plt.xlabel("Person ID")
plt.ylabel("Sai số tái dựng (MSE)")
plt.xticks(range(1, 7))
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

psnr_list = []
ssim_list = []
person_ids = []

for person_id in range(1, 16):
    try:
        # Load ảnh gốc
        fn = path + prefix + str(person_id).zfill(2) + '.centerlight' + suffix
        img = Image.open(fn).convert('L')
        img = img.resize((98, 116))
        im = np.array(img)

        # reshape và chuẩn hóa
        x = im.reshape(D, 1) - pca.mean_.reshape(D, 1)
        z = U.T.dot(x)
        x_tilde = U.dot(z) + pca.mean_.reshape(D, 1)
        im_tilde = x_tilde.reshape(116, 98)

        # Chuyển về uint8 để đánh giá
        im_uint8 = im.astype(np.uint8)
        im_tilde_uint8 = np.clip(im_tilde, 0, 255).astype(np.uint8)

        # Tính PSNR và SSIM
        psnr = peak_signal_noise_ratio(im_uint8, im_tilde_uint8)
        ssim = structural_similarity(im_uint8, im_tilde_uint8)

        psnr_list.append(psnr)
        ssim_list.append(ssim)
        person_ids.append(person_id)

        print(f'Person {person_id}, PSNR: {psnr:.2f}, SSIM: {ssim:.4f}')

    except Exception as e:
        print(f"Loi voi person {person_id}: {e}")

# VẼ BIỂU ĐỒ 

plt.figure(figsize=(12, 5))

# PSNR
plt.subplot(1, 2, 1)
plt.bar(person_ids, psnr_list, color='skyblue')
plt.title('PSNR (Peak Signal-to-Noise Ratio)')
plt.xlabel('Person ID')
plt.ylabel('PSNR (dB)')
plt.ylim(0, max(psnr_list) + 5)

# SSIM
plt.subplot(1, 2, 2)
plt.bar(person_ids, ssim_list, color='salmon')
plt.title('SSIM (Structural Similarity)')
plt.xlabel('Person ID')
plt.ylabel('SSIM (0-1)')
plt.ylim(0, 1.05)

plt.tight_layout()
plt.show()


In [ ]:
from PIL import Image

cnt = 0 
for person_id in [10]:
    for ii, state in enumerate(states):
        fn = path + prefix + str(person_id).zfill(2) + '.' + state + suffix
        img = Image.open(fn).convert('L')
        # resize ve kich thuoc dong nhat (neu can)
        img = img.resize((98, 116))  
        im = np.array(img)

        f1 = plt.imshow(im, interpolation='nearest')
        f1.axes.get_xaxis().set_visible(False)
        f1.axes.get_yaxis().set_visible(False)

        fn = 'ex' + str(ii).zfill(2) +  '.png'
        plt.axis('off')
        plt.savefig(fn, bbox_inches='tight', pad_inches=0)
        plt.show()
